In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import datetime
from keras.models import load_model
import csv
from keras.models import Sequential
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import metrics
from keras.applications.resnet50 import ResNet50, preprocess_input
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


## transfer learning 
1- train top layer with 128x128 (2 epochs)

2- train the whole network with 128x128 (10 epochs)


In [2]:
#Parameters
batch_size = 32 
top_epochs = 2

output_size = 3
classes = ['normal', 'pneumonia', 'COVID-19']

target_size= (128, 128)

train_size = 37974
test_size = 4235
train_step = np.ceil(train_size/batch_size) 
test_step = 2  
verbose = 1

In [3]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

base_model = ResNet50(weights='imagenet', include_top=False)


# add a top layer
x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), padding="same", activation="relu")(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(output_size, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


C:\Users\rashid\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
# train top layers only (128x128)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(datagen.flow_from_directory('data/train', classes=classes, class_mode='categorical',target_size=target_size), 
            steps_per_epoch=train_step, epochs=top_epochs, verbose=verbose, 
            validation_data=test_datagen.flow_from_directory('data/test', classes=classes, class_mode='categorical',target_size=target_size),
            validation_steps=test_step)



Found 37974 images belonging to 3 classes.
Found 4235 images belonging to 3 classes.
Epoch 1/2
1187/1187 [==============================] - 631s 531ms/step - loss: 0.4807 - acc: 0.8195 - val_loss: 1.1370 - val_acc: 0.6094
Epoch 2/2
1187/1187 [==============================] - 613s 516ms/step - loss: 0.3911 - acc: 0.8607 - val_loss: 1.2690 - val_acc: 0.6094


In [5]:
#Train with 128x128 full network
epochs= 10
N=0
    
for layer in model.layers[:N]:
    layer.trainable = False
for layer in model.layers[N:]:
    layer.trainable = True


model.compile('adam', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(datagen.flow_from_directory('data/train', classes=classes, class_mode='categorical',target_size=target_size), 
                    steps_per_epoch=train_step, epochs=epochs, verbose=verbose, 
                    validation_data=test_datagen.flow_from_directory('data/test', classes=classes, class_mode='categorical',target_size=target_size),
                    validation_steps=test_step, use_multiprocessing=False)

model.save('models/ResNet128.h5')


Found 37974 images belonging to 3 classes.
Found 4235 images belonging to 3 classes.
Epoch 1/10
1187/1187 [==============================] - 689s 581ms/step - loss: 0.5877 - acc: 0.7922 - val_loss: 0.6757 - val_acc: 0.6719
Epoch 2/10
1187/1187 [==============================] - 687s 579ms/step - loss: 0.4055 - acc: 0.8563 - val_loss: 1.1105 - val_acc: 0.6875
Epoch 3/10
1187/1187 [==============================] - 691s 582ms/step - loss: 0.3743 - acc: 0.8718 - val_loss: 0.3859 - val_acc: 0.9062
Epoch 4/10
1187/1187 [==============================] - 662s 558ms/step - loss: 0.4130 - acc: 0.8514 - val_loss: 0.3563 - val_acc: 0.8906
Epoch 5/10
1187/1187 [==============================] - 648s 546ms/step - loss: 0.3060 - acc: 0.8883 - val_loss: 0.6474 - val_acc: 0.7031
Epoch 6/10
1187/1187 [==============================] - 645s 543ms/step - loss: 0.3337 - acc: 0.8766 - val_loss: 0.7510 - val_acc: 0.6406
Epoch 7/10
1187/1187 [==============================] - 648s 546ms/step - loss: 0.3499 